In [13]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generative AI Knowledge Base model predictions

To run this notebook, make sure you have uploaded at least one document into your knowledge base.

> ⭐️ If you haven't, follow the [**Uploading documents and query model** tutorial](https://console.cloud.google.com/products/solutions/deployments?walkthrough_id=panels--sic--generative-ai-knowledge-base_toc).

Before you begin, make sure all the dependencies are installed.

In [ ]:
!pip install google-genai google-cloud-aiplatform google-cloud-firestore

## Overview

A **Large Language Model (LLM)** can be very good at answering general questions.
But it might not do as well to answer questions from your documents on its own.

The LLM will answer only from what it learned from its _training dataset_.
Your documents might include information or words that weren't on that dataset.
Or they might be used in a different or more specialized context.

This is where **Vector Search** comes into place.
Each time you upload a document, the Cloud Function webhook processes it.
When a document is processed, each individual page is _indexed_.
This allows us to not only find documents, but the specific pages.

The relevant pages can then be used as _context_ for the LLM to answer the question.
This _grounds_ the model to answer questions based on the documents only.
Without this, the model might give wrong answers, or _hallucinations_.

## My Google Cloud resources

Fill in your project ID, the
[Google Cloud location](https://cloud.google.com/about/locations)
you want to use, and your
Vector Search index endpoint ID.
If you followed the tutorial, the deployed index ID should be `deployed_index`, otherwise change it to the ID you chose.

You can find your Vector Search index endpoint ID in the [Index endpoints tab](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints).

> 💡 The Vector Search index endpoint ID looks like a number, like `1234567890123456789`.

Run the following cell to set up your resources and authenticate to your account.

In [ ]:
# @title
from google.colab import auth

project_id = "" # @param {type:"string"}
location = "us-central1" # @param {type:"string"}
index_endpoint_id = "" # @param {type:"string"}
deployed_index_id = "deployed_index" # @param {type:"string"}

assert project_id, "Please set the project_id"
assert location, "Please set the location"
assert index_endpoint_id, "Please set the index_endpoint_id"
assert deployed_index_id, "Please set the deployed_index_id"

auth.authenticate_user(project_id=project_id)

The first step is to initialize the GenAI client library using the project and location of your choice.

In [ ]:
from google import genai

genai_client = genai.Client(vertexai=True, project=project_id, location=location)

## Get text embeddings

You can use the Gecko model to get embeddings from text.
For more information, see the
[Get text embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings)
page.

In [ ]:
def get_text_embedding(text: str) -> list[float]:
    response = genai_client.models.embed_content(
        model="text-embedding-005",
        contents=[text],
    )
    embeddings = response.embeddings or []
    assert len(embeddings) == 1, f"expected 1 embedding, got {len(embeddings)}"
    return embeddings[0].values or []

# Convert the question into an embedding.
question = "What are LFs and why are they useful?"
question_embedding = get_text_embedding(question)
print(f"Embedding dimensions: {len(question_embedding)}")

## Find document context

All the documents you have processed have been indexed into your Vector Search index.
You can query for the closest embeddings to a given embedding from your Vector Search index endpoint.

> 💡 If you haven't processed any documents yet, you won't get any results.

In [ ]:
from google.cloud import aiplatform
from itertools import groupby

aiplatform.init(project=project_id, location=location)

def find_document(question: str, index_endpoint_id: str, deployed_index_id: str) -> tuple[str, int]:
    # Get embeddings for the question.
    embedding = get_text_embedding(question)

    # Find the closest point from the Vector Search index endpoint.
    endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_id)
    point = endpoint.find_neighbors(
        deployed_index_id=deployed_index_id,
        queries=[embedding],
        num_neighbors=1,
    )[0][0]

    # Get the document name and page number from the point ID.
    (filename, page_number) = point.id.split(':', 1)
    return (filename, int(page_number))

# Query the Vector Search index for the most relevant page.
(filename, page_number) = find_document(question, index_endpoint_id, deployed_index_id)
print(f"{filename=} {page_number=}")

## Get document text

When documents were processed, their text was stored in Firestore as well.
The Vector Search query returned the relevant documents with their page numbers.
With this you can download the document's pages and give only the most relevant page to the model.

In [ ]:
from google.cloud import firestore

def get_document_text(filename: str, page_number: int) -> str:
    db = firestore.Client(database="knowledge-base-database")
    doc = db.collection("documents").document(filename.replace('/', '-'))
    return doc.get().get('pages')[page_number]

# Download the document's page text from Firestore.
context = get_document_text(filename, page_number)
print(f"{context[:1000]}\n...\n...")

## Ask a foundational model

With the relevant context ready, you can now make a _prompt_ that includes both the context and the question.

Here's Gemini's response.
Note that Gemini responds in [Markdown](https://www.markdownguide.org).

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import GenerateContentConfig

def ask_model(question: str) -> None:
    (filename, page_number) = find_document(question, index_endpoint_id, deployed_index_id)
    context = get_document_text(filename, page_number)
    response = genai_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=question,
        config=GenerateContentConfig(
            system_instruction=[
                "Answer the question based on the following text:",
                context,
            ],
        ),
    )
    print(question)
    display(Markdown(response.text))

ask_model("What are LFs and why are they useful?")

## (Optional) Ask your tuned model

If you want to tune a model, follow the [**Fine-tune an LLM model** tutorial](https://console.cloud.google.com/products/solutions/deployments?walkthrough_id=panels--sic--generative-ai-knowledge-base_toc).

First, find the tuning job ID for your tuned model.

In [ ]:
from vertexai.tuning import sft

for tuning_job in sft.SupervisedTuningJob.list():
    model_name = tuning_job.gca_resource.tuned_model_display_name
    tuning_job_id = tuning_job.resource_name
    print(f"{model_name}: {tuning_job_id}")

Copy your tuning job ID and paste it below.
Don't forget to run the cell to define the `tuning_job_id` variable.

In [ ]:
tuning_job_id = "" # @param {type:"string"}

assert tuning_job_id, "Please set the tuning_job_id"

In [ ]:
from vertexai.tuning import sft

tuning_job = sft.SupervisedTuningJob(tuning_job_id)
assert tuning_job.has_ended, "Please wait until the tuning job finishes."
assert tuning_job.tuned_model_endpoint_name

tuned_model_endpoint = tuning_job.tuned_model_endpoint_name
print(f"{tuned_model_endpoint=}")
# The tuned model endpoint follows this format:
#   projects/<PROJECT_NUMBER>/locations/<LOCATION>/endpoints/<MODEL_ENDPOINT_ID>

In [ ]:
from vertexai.generative_models import GenerativeModel

def ask_tuned_model(tuned_model_endpoint: str, question: str) -> None:
    (filename, page_number) = find_document(question, index_endpoint_id, deployed_index_id)
    context = get_document_text(filename, page_number)
    response = genai_client.models.generate_content(
        model=tuned_model_endpoint,
        contents=[f"Text: {context}", question],
        config=GenerateContentConfig(
            system_instruction=[
                "Answer the question based on the following text",
            ],
        ),
    )
    print(question)
    display(Markdown(response.text))

ask_tuned_model(tuned_model_endpoint, "What are LFs and why are they useful?")